© Валерий Студенников, курс "Инструменты анализа данных"

# Предобработка данных

**Предобработка данных** — это действия с данными перед передачей их модели, могут включать в себя центрирование, нормализацию, сдвиг, вращение, обрезку. Часто требуется при необходимости очистить данные от ненужного или дополнить какой-то информацией.

## Виды проблем в данных:

**Неполные данные**: заказчик выслал не все данные, в них есть пропуски, нет данных за определённый период, есть данные только для части задач. Например, есть рейсы только одной авиакомпании, есть суммы покупок с января по март и с июня по декабрь.

**Грязные данные**:
- данные в плохом или разном формате;
- в данных есть мусор, например, смешиваются русские и английские буквы, данные в виде шифра,
- разные меры данных, например, часть в метрах, часть в футах; 
- данные старые, значения признака перемешиваются.

**Плохая разметка**: в данных есть ошибки, разная оценка данных из-за человеческого фактора, разметка не соответствует реальности, размеченных данных очень много — должна быть «золотая середина». Чем лучше хотите получить разметку, тем больше времени придётся потратить.

**Данные с утечкой**: в данных нужно избегать признаков, от которых явно зависит целевая переменная. Такая ситуация приводит к тому, что во время обучения модель всё предсказывает, но в боевых условиях бесполезна.

## Работа с пропусками

**Пропуски** — отсутствие значений в данных. Обычно это случается из-за ошибок человека: забыли, какие данные не внесли, не всё перенесли из другого документа, неверно заполнили документ, случайно удалили часть данных. Например, при опросе респонденты отвечали только на часть вопросов, не указывая какие-то данные.

Что делать с пропусками?

- **Удалить пропуски**: удалить строки с пропусками, если таких строк немного, или столбцы (признаки) в данных, если их много, и это не повлияет на результат в целом.
- **Заменить пропуски**: заменить на среднее значение (медианой) или на самый часто попадающийся вариант. Например, опрос респондентов проходил в Москве, вероятнее, большинство — жители города, а не туристы. Значит, в графе город можно поставить «Москва». Или же по другим данным в документе можно восстановить пол людей, которые его не указали.
- **Записать новое значение**: например, вместо пропуска в столбце «Пол» указать «не определён».
- **Заменить алгоритмами МО**: признак можно считать целевой переменной и обучать модель, чтобы предсказать пропущенные значения.

In [2]:
import numpy as np
import pandas as pd

In [3]:
test_data = pd.DataFrame(
    [[1, 2, np.nan], [3, np.nan, 4],
    [0, 1, 2]], columns=['one', 'two', 'three']
)

test_data

,one,two,three
0,1,2.0,NaN
1,3,NaN,4.0
2,0,1.0,2.0


### Поиск пропусков

В данных часто бывают пропуски. В Pandas их можно находить с помощью метода `.isna()`:

In [9]:
test_data.isna()
# В местах пропусков у нас появится значение True.

,one,two,three
0,False,False,True
1,False,True,False
2,False,False,False


Можно находить пропуски в пределах одного признака (столбца):

In [12]:
test_data['two'].isna()

0    False
1     True
2    False
Name: two, dtype: bool

### Удаление столбцов/строк с пропусками

С помощью метода `.dropna()` можно удалять столбцы/строки с пропусками, указывая ось с помощью параметра `axis`.  
Если нужно удалить строки, в которых встречается пропуск (NaN), `axis=0`:

In [13]:
test_data.dropna(axis=0)

,one,two,three
2,0,1.0,2.0


Если нужно удалить столбцы, в которых встречается пропуск (NaN), `axis=1`:

In [14]:
test_data.dropna(axis=1)

,one
0,1
1,3
2,0


По умолчанию, параметр `axis` равен 0:


Если нужно удалить пропуски только для определенных столбцов (или строк, при указании другого значения параметра axis), нужно передать список индексов (номеров строк или названий столбцов) столбцов (или строк) в параметр subset:

In [15]:
test_data.dropna(subset=[0], axis=1)

,one,two
0,1,2.0
1,3,NaN
2,0,1.0


In [16]:
test_data.dropna(subset=['two'], axis=0)

,one,two,three
0,1,2.0,NaN
2,0,1.0,2.0


### Заполнение статистиками

Иногда удаление строк/столбцов с пропусками - слишком грубое решение. Можно заменять пропуски статистиками или каким-то константным значением с помощью метода `fillna()`:

In [17]:
# По умолчанию заменяет все пропуски
test_data.fillna(-1)

,one,two,three
0,1,2.0,-1.0
1,3,-1.0,4.0
2,0,1.0,2.0


Можно заменять разными статистиками, например:
 - среднее значение, `np.mean()`
 - максимальное значение, `np.max()`
 - минимальное значение, `np.min()`
 - медиана, `np.median()`

In [20]:
test_data['two'].fillna( test_data.two.max() )

0    2.0
1    2.0
2    1.0
Name: two, dtype: float64

In [21]:
test_data['three'].fillna( test_data['three'].mean() )

0    3.0
1    4.0
2    2.0
Name: three, dtype: float64

## "Нормализация" признаков

Некоторые алгоритмы чувствительны к **масштабу признаков**: если масштаб разный — алгоритмы хуже сходятся к оптимальным значениям. Один из способов обработки данных — **нормализация** — процесс приведения данных к одному масштабу, обеспечивает лучшие условия выборки, изменения, удаления данных.

**Базовые методы нормализации**:

- **MinMax** нормализация — приведение данных к масштабу между 0 и 1.
- **Стандартная** нормализация — данные имеют среднее 0 и стандартное отклонение 1.

Иногда данные лучше удалить, если они не несут пользы. Например, столбец считался по ошибке и содержит мусор, или признак не имеет полезной информации, или столбец содержит всегда одинаковое значение.


In [ ]:
import sklearn.preprocessing
scaler = sklearn.preprocessing.MinMaxScaler()
scaler = sklearn.preprocessing.StandardScaler()

scaler.fit(data)
data_transformed = scaler.fit_transform( data )

`MinMaxScaler` и `StandardScaler` сохраняют параметры, с которыми проводят нормализацию. Поэтому после нормализации признаков в тренировочной выборке нужно будет применить ту же нормализацию с валидационными и тестовыми данными.

In [ ]:
new_data_transformed = scaler.transform( new_data )

### Преобразование признаков, имеющих не нормальное распредление

Один из способов визуализации — **гистограмма** — вид диаграммы, показывающий данные в форме столбцов. Демонстрирует распределение данных по выборке внутри столбца и значений в признаке. Позволяет увидеть проблемы или некорректные значения в данных, преобразовать их в наглядную форму и распределить, чтобы облегчить работу.

In [ ]:
df.feature.hist()

**Графики Q-Q** (квантиль-квантиль) сравнивают два распределения вероятностей путем построения совместных квантилей. График Q-Q используется для сравнения форм распределений, обеспечивая графическое представление того, насколько подобны или различны положение, масштаб и асимметрия двух распределений.

In [ ]:
stats.probplot( iris.sepal_length, dist="norm", plot=pylab)

Многие алгоритмы работают лучше, когда на вход принимают нормально распределённые данные. Один из способов сделать данные «нормальными» — взять **логарифм**. При этом не должно быть значений равных 0, иначе метод `np.log` выдаст бесконечность.

Если значения всё-таки начинаются с 0, то перед тем, как взять логарифм, можно прибавить к данным 1.

In [ ]:
df.balance = np.log( df.balance )

Другой способ — взять квадратный корень от данных:

In [ ]:
df.balance = np.sqrt( df.balance )

### Скореллированные признаки

**Correlation plot** — корреляционный график. Признаки в данных могут иметь сильную корреляцию, то есть линейную зависимость. Если зависимость большая, признаки несут избыточную информацию, поэтому скореллированные признаки лучше удалять.

In [ ]:
import seaborn

corr_matrix = df.drop(['categirial','features'], axis=1).corr()

seaborn.heatmap( corr_matrix, cmap = 'seismic', center = 0, ax = ax )

### Feature Engineering

Feature Engineering — способ создания признаков, техника решения задач МО, которая увеличивает качество разрабатываемых алгоритмов. Превращает специфичные данные в понятные для модели векторы.

Способы:

* **Ручное создание признаков**  
Новые признаки можно создавать на основе имеющихся, если понятно, что означает тот или иной признак. Техника может значительно улучшить модель, но требуется погружение в предметную область. Например, зная массу и рост человека, можно посчитать индекс массы тела; зная координаты места нарушения ПДД и центра города, можно посчитать расстояние между ними.
* **Полиномиальные признаки**  
Например, попарное перемножение имеющихся столбцов, чтобы получить нелинейные комбинации признаков. Другой пример — возведение признаков в квадрат или корень из признака, чтобы получить искусственные признаки.
* **Dummy-переменные**  
Между категориальными признаками не всегда есть порядок, поэтому их следует заменять на dummy-переменные, в виде цифр, чтобы алгоритм подумал, что в данных есть порядок. Например, лев, тигр, медведь лучше записать как 0, 1, 2. Расстояние между «лев» и «тигр» равно 1, а между «лев» и «медведь» — 2, хотя это не так. Решение — создать новые признаки по количеству категорий. Для признака «лев» значение будет 1, если это лев, иначе 0. Теперь расстояния между всеми признаками равны 1.
* **Работа с датой/временем**  
Обычно дата — это строка, похожая на «2005-06-02». В таком виде признак имеет мало смысла, но его можно разделить на набор других признаков. Модель не принимает текст, поэтому заменяем его на набор других числовых признаков. Например, день, месяц, год или день недели, праздник/выходной.
* **Статистики по наборам признаков**  
Если в данных есть признаки, описывающие значение в разные моменты времени, то они могут быть полезными. Например, зарплата_2016, зарплата_2017, зарплата_2018. Отсюда можно вытащить среднюю зарплату за 3 года, медианную зарплату, динамику зарплаты, максимальную и минимальную зарплату.